In [1]:
%pip install langchain transformers neo4j faiss-cpu pinecone-client neo4j Neo4jGraph pandas


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement Neo4jGraph (from versions: none)
ERROR: No matching distribution found for Neo4jGraph


In [2]:
dBurl = "neo4j+s://b26b71fa.databases.neo4j.io"
password = "YWpXBbSXHIcloDAhR_ZfTo9iR6jLzFIwyux_bNBU6lA"
username = "neo4j"

In [4]:
from langchain.graphs import Neo4jGraph
from neo4j import GraphDatabase
import requests

graph = Neo4jGraph(dBurl, username, password)
driver = GraphDatabase.driver(dBurl, auth=(username, password))

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

# Load embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
documents = ["This is a test document", "This is another test document"]
faiss_store = FAISS.from_texts(documents, embedder)

# from langchain.vectorstores.neo4j_vector import Neo4jVector
# from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# vector_index = Neo4jVector.from_existing_graph(
#     hf_embeddings,
#     url = dBurl,
#     username = username,
#     password = password,
#     index_name='tasks',
#     node_label="Tasks",
#     text_node_properties=['name', 'description', 'status'],
#     embedding_node_property="embedding"
# )

c:\Users\prave\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\prave\AppData\Local\Temp\ipykernel_10952\3111894771.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\prave\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior w

In [7]:
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

def hybrid_retrieval(query):
    # First try retrieving from the knowledge graph
    kg_result = run_query(f"MATCH (n) WHERE n.name CONTAINS '{query}' RETURN n LIMIT 1")
    
    if kg_result:
        return kg_result[0]  # If found in KG
    
    # If not, fall back to vector search
    docs = faiss_store.similarity_search(query)
    return docs[0].page_content if docs else None

responses = faiss_store.similarity_search("How will RecommendationService be updated?")
print(responses)


[Document(page_content='This is another test document'), Document(page_content='This is a test document')]


In [5]:
from transformers import pipeline

# Load a language model
generator = pipeline("text-generation", model="gpt2")

# Function to generate a response using retrieved context
def generate_response(query):
    context = hybrid_retrieval(query)
    prompt = f"Answer the question based on the following context:\n{context}\n\nQuestion: {query}"
    
    response = generator(prompt, max_length=100, num_return_sequences=1)
    return response[0]["generated_text"]


In [ ]:
print("Welcome to the Knowledge Graph-based RAG Chatbot!")
query = "How many open tickets there are?"
response = generate_response(query)
print(f"Bot: {response}")